## DO NOT RUN 
UNLESS YOU WANT MORE THEN 10 000 FILES GENERATED

In [ ]:
%%bash
ls data/movielens

In [ ]:
%%bash
head data/movielens/users.csv

In [1]:
import pandas as pd

In [2]:
path = "data/movielens/"
users = pd.read_csv(path+"users.csv", sep=";")
ratings = pd.read_csv(path+"ratings.csv", sep=";")
movies = pd.read_csv(path+"movies.csv", sep=";", encoding="latin")

In [3]:
for df in [users, ratings, movies]:
    print(df.head())
    print("++++++++++++++++++++")

   userId gender  age  occupation zip-code
0       1      F    1          10    48067
1       2      M   56          16    70072
2       3      M   25          15    55117
3       4      M   45           7     2460
4       5      M   25          20    55455
++++++++++++++++++++
   userId  movieId  rating  timestamp
0       1     1193       5  978300760
1       1      661       3  978302109
2       1      914       3  978301968
3       1     3408       4  978300275
4       1     2355       5  978824291
++++++++++++++++++++
   movieId                               title                        genres  \
0        1                    Toy Story (1995)   Animation|Children's|Comedy   
1        2                      Jumanji (1995)  Adventure|Children's|Fantasy   
2        3             Grumpier Old Men (1995)                Comedy|Romance   
3        4            Waiting to Exhale (1995)                  Comedy|Drama   
4        5  Father of the Bride Part II (1995)                        Co

In [4]:
data = pd.merge(pd.merge(ratings, users), movies)

In [5]:
data.drop('Unnamed: 3', axis=1, inplace=True)

In [6]:
data.head()

,userId,movieId,rating,timestamp,gender,age,occupation,zip-code,title,genres
0,1,1193,5,978300760,F,1,10,48067,One Flew Over the Cuckoo's Nest (1975),Drama
1,1,661,3,978302109,F,1,10,48067,James and the Giant Peach (1996),Animation|Children's|Musical
2,1,914,3,978301968,F,1,10,48067,My Fair Lady (1964),Musical|Romance
3,1,3408,4,978300275,F,1,10,48067,Erin Brockovich (2000),Drama
4,1,2355,5,978824291,F,1,10,48067,"Bug's Life, A (1998)",Animation|Children's|Comedy


In [8]:
print(len(data))

1000209


In [7]:
data.iloc[0]

userId                                             1
movieId                                         1193
rating                                             5
timestamp                                  978300760
gender                                             F
age                                                1
occupation                                        10
zip-code                                       48067
title         One Flew Over the Cuckoo's Nest (1975)
genres                                         Drama
Name: 0, dtype: object

In [9]:
%pip install fpdf

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: C:\Users\farah\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [15]:
from fpdf import FPDF
import os
import math

In [39]:
output_dir = "pdf_exports_as_it_is"
os.makedirs(output_dir, exist_ok=True)  # create a directory to store the PDFs

In [40]:
# Converts timestamp to readable format
from datetime import datetime


def format_timestamp(ts):
    return datetime.utcfromtimestamp(int(ts)).strftime('%B %d, %Y')

In [42]:
age_dict = {
    1: "under 18", 18: "between 18 and 24", 25: "between 25 and 34",
    35: "between 35 and 44", 45: "between 45 and 49",
    50: "between 50 and 55", 56: "56 or older"
}

occupation_dict = {
    0: "unspecified occupation", 1: "academic or educator", 2: "artist", 3: "clerical or administrative worker",
    4: "college or graduate student", 5: "customer service representative", 6: "healthcare professional",
    7: "executive or managerial worker", 8: "farmer", 9: "homemaker", 10: "K-12 student",
    11: "lawyer", 12: "programmer", 13: "retired individual", 14: "sales or marketing professional",
    15: "scientist", 16: "self-employed worker", 17: "technician or engineer",
    18: "tradesperson or craftsman", 19: "unemployed individual", 20: "writer"
}

def make_sentence(row):
    date_str = format_timestamp(row["timestamp"])
    gender_str = "male" if row["gender"] == "M" else "female"
    age_range = age_dict.get(int(row["age"]), "unknown age range")
    occupation = occupation_dict.get(int(row["occupation"]), "unspecified occupation")
    zip_code = row["zip-code"]
    title = row["title"]
    genres = row["genres"].replace('|', ', ')
    rating = row["rating"]

    return (
        f"On {date_str}, a {gender_str} user aged {age_range}, employed as a {occupation} "
        f"and residing in ZIP code {zip_code}, rated the film \"{title}\" ({genres}) "
        f"with a score of {rating} on the MovieLens platform. "
    )

In [43]:
print(make_sentence(data.iloc[0]))

On December 31, 2000, a female user aged under 18, employed as a K-12 student and residing in ZIP code 48067, rated the film "One Flew Over the Cuckoo's Nest (1975)" (Drama) with a score of 5 on the MovieLens platform. 


In [44]:
def create_pdf_from_dataframe(df, output_dir):
    """creates a PDF file for each 100 rows in the DataFrame.
        inputs:
        - df: pandas DataFrame containing the data
        - output_dir: directory where the PDFs will be saved
        outputs:
        - None, but creates PDF files in the specified directory
    """    
    chunk_size = 100  # number of rows per PDF
    num_chunks = math.ceil(len(df) / chunk_size)

    for chunk_index in range(num_chunks):
        start = chunk_index * chunk_size
        end = min((chunk_index + 1) * chunk_size, len(df))
        chunk = df.iloc[start:end]

        pdf = FPDF()
        pdf.add_page()
        pdf.set_auto_page_break(auto=True, margin=15)
        pdf.set_font("Arial", size=12)

        pdf.cell(0, 10, f"MovieLens summary report - chunk #{chunk_index + 1}", ln=True, align='C')
        pdf.ln(10)  # add space

        # Add fields one by one
        for _, row in chunk.iterrows():
            text = make_sentence(row)
            pdf.multi_cell(0, 10, text)
            pdf.ln(5)

        # Name the file based on row info
        filename = f"movies_chunk{chunk_index +1}.pdf"
        filepath = os.path.join(output_dir, filename)
        pdf.output(filepath)

    print("PDFs created in:", output_dir)

In [45]:
create_pdf_from_dataframe(data, output_dir)
# Check the output directory

PDFs created in: pdf_exports_as_it_is
